In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [51]:
def get_retro_teams():
    teams = [item.strip() for item in open("all_teams_retro.txt",'r')]
    return teams

teams = get_retro_teams()

In [52]:
def get_stats(s):
    """
    parameter: soup object that contains the scraped retrosheet team webpage
    Return: string containing the values I want scraped"""
    html = list(s.children)[2]
    body = list(html.children)[5]
    data = list(body.children)[12]
    line = list(data.children)[-1]
    string = line.split()
    #main lines that I want from the webpage, which contain the AVG, OBP, SLG and BFW values
    total = string[:24]
    oppon = string[25:48]
    team_stats = np.concatenate(([[float(i) for i in total[-4:]], [float(i) for i in oppon[-3:]]]))
    return team_stats 

def get_values(team,year,df):
    """
    return row in dataframe that matches team and year keys"""
    return np.ravel(df_yr[df_yr['Team Abbreviation'] == team][df_yr['Year'] == year].values)

In [53]:
get_values('WSN',2012,df_yr)

C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


array(['Washington Nationals', 'WSN', 2012, 'National', 'East', 162, 98,
       64, 0, 0.605, '-', 731, 594], dtype=object)

In [54]:
df= pd.read_csv("team_abbrev.csv",index_col=1)
df.head()

,New
Retro,
ARI,ARI
ATL,ATL
BAL,BAL
BOS,BOS
CHN,CHC


In [ ]:
team_dict = df.to_dict()['New']
team_dict['NYN']

In [56]:
df_yr = pd.read_csv('2012-2019_team_records.csv')
df_yr

,Team Name,Team Abbreviation,Year,League,Division,Games,Wins,Loses,Ties,Win_Percentage,Games_Back,Runs_Scored,Runs_Allowed
0,Washington Nationals,WSN,2012,National,East,162,98,64,0,0.605,-,731,594
1,Atlanta Braves,ATL,2012,National,East,162,94,68,0,0.580,4,700,600
2,Philadelphia Phillies,PHI,2012,National,East,162,81,81,0,0.500,17,684,680
3,New York Mets,NYM,2012,National,East,162,74,88,0,0.457,24,650,709
4,Miami Marlins,MIA,2012,National,East,162,69,93,0,0.426,29,609,724
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Houston Astros,HOU,2019,American,West,162,107,55,0,0.660,-,920,640
236,Oakland Athletics,OAK,2019,American,West,162,97,65,0,0.599,10,845,680
237,Texas Rangers,TEX,2019,American,West,162,78,84,0,0.481,29,810,878
238,Anaheim Angels,LAA,2019,American,West,162,72,90,0,0.444,35,769,868


In [57]:
df_yr.columns.values

array(['Team Name', 'Team Abbreviation', 'Year', 'League', 'Division',
       'Games', 'Wins', 'Loses', 'Ties', 'Win_Percentage', 'Games_Back',
       'Runs_Scored', 'Runs_Allowed'], dtype=object)

In [58]:
stats = pd.DataFrame(columns=np.concatenate((df_yr.columns.values, ['AVG','OBP','SLG','BFW','O_AVG','O_OBP','O_SLG']),axis=0) )
stats

,Team Name,Team Abbreviation,Year,League,Division,Games,Wins,Loses,Ties,Win_Percentage,Games_Back,Runs_Scored,Runs_Allowed,AVG,OBP,SLG,BFW,O_AVG,O_OBP,O_SLG


In [59]:
index = 0
for year in list(range(2012,2020)):
    print(year)
    for team in teams:                 
        #print(team, year)
        page = requests.get(f"https://www.retrosheet.org/boxesetc/{year}/U{team}0{year}.htm")
        soup = BeautifulSoup(page.content, 'html.parser')
        stats.loc[index] = np.concatenate((get_values(team_dict[team],year,df_yr), list((get_stats(soup)))))
        index += 1

2012


C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


2013
2014
2015
2016
2017
2018
2019


In [60]:
stats

,Team Name,Team Abbreviation,Year,League,Division,Games,Wins,Loses,Ties,Win_Percentage,Games_Back,Runs_Scored,Runs_Allowed,AVG,OBP,SLG,BFW,O_AVG,O_OBP,O_SLG
0,Atlanta Braves,ATL,2012,National,East,162,94,68,0,0.580,4,700,600,0.247,0.320,0.389,3.2,0.243,0.306,0.378
1,Washington Nationals,WSN,2012,National,East,162,98,64,0,0.605,-,731,594,0.261,0.322,0.428,5.0,0.237,0.303,0.373
2,Philadelphia Phillies,PHI,2012,National,East,162,81,81,0,0.500,17,684,680,0.255,0.317,0.400,2.8,0.251,0.306,0.407
3,New York Mets,NYM,2012,National,East,162,74,88,0,0.457,24,650,709,0.249,0.316,0.386,-2.5,0.251,0.315,0.401
4,Miami Marlins,MIA,2012,National,East,162,69,93,0,0.426,29,609,724,0.244,0.308,0.382,-5.4,0.263,0.327,0.399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Houston Astros,HOU,2019,American,West,162,107,55,0,0.660,-,920,640,0.274,0.352,0.495,12.8,0.221,0.283,0.397
236,Oakland Athletics,OAK,2019,American,West,162,97,65,0,0.599,10,845,680,0.249,0.327,0.448,1.6,0.242,0.308,0.404
237,Anaheim Angels,LAA,2019,American,West,162,72,90,0,0.444,35,769,868,0.247,0.324,0.422,-3.2,0.254,0.331,0.459
238,Seattle Mariners,SEA,2019,American,West,162,68,94,0,0.420,39,758,893,0.237,0.316,0.424,-3.6,0.263,0.326,0.469


In [61]:
#stats.to_csv("2012-2019_team_player_stats.csv",index=False)